### Imports

In [1]:
import pandas as pd

In [2]:
# pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.set_option('display.max_columns', None)

### Leemos y consolidamos los datasets

In [3]:
ventas = pd.read_csv('../Datasets/sell-in.txt', sep='\t')
productos = pd.read_csv('../Datasets/tb_productos.txt', sep='\t')
stock = pd.read_csv('../Datasets/tb_stocks.txt', sep='\t')
productos_predecir = pd.read_csv('../Datasets/productos_a_predecir.csv')

ventas_grouped = ventas.groupby(['periodo', 'customer_id', 'product_id']).agg({
    'plan_precios_cuidados': 'sum',
    'cust_request_qty': 'sum',
    'cust_request_tn': 'sum',
    'tn': 'sum'
}).reset_index()

ventas_producto = pd.merge(ventas_grouped, productos, on=['product_id'], how='inner')
final_dataset = pd.merge(ventas_producto, stock, on=['product_id', 'periodo'], how='left')
final_dataset = pd.merge(final_dataset, productos_predecir, on=['product_id'], how='inner')

final_dataset.to_csv('../Datasets/final_dataset_by_client.csv', sep='\t', index=False)

In [4]:
final_dataset.isnull().sum()

periodo                        0
customer_id                    0
product_id                     0
plan_precios_cuidados          0
cust_request_qty               0
cust_request_tn                0
tn                             0
cat1                           0
cat2                           0
cat3                           0
brand                          0
sku_size                       0
stock_final              1347500
dtype: int64

### Analisis exploratorio basico

#### Analisis inicial

In [5]:
final_dataset.shape

(2332211, 13)

In [6]:
final_dataset.head(20)

,periodo,customer_id,product_id,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn,cat1,cat2,cat3,brand,sku_size,stock_final
0,201701,10001,20001,0,11,99.43861,99.43861,HC,ROPA LAVADO,Liquido,ARIEL,3000,NaN
1,201701,10002,20001,0,17,38.68301,35.72806,HC,ROPA LAVADO,Liquido,ARIEL,3000,NaN
2,201701,10003,20001,0,17,143.49426,143.49426,HC,ROPA LAVADO,Liquido,ARIEL,3000,NaN
3,201701,10004,20001,0,9,184.72927,184.72927,HC,ROPA LAVADO,Liquido,ARIEL,3000,NaN
4,201701,10005,20001,0,23,19.08407,19.08407,HC,ROPA LAVADO,Liquido,ARIEL,3000,NaN
5,201701,10006,20001,0,49,43.83179,43.83179,HC,ROPA LAVADO,Liquido,ARIEL,3000,NaN
6,201701,10007,20001,0,3,65.12087,65.12087,HC,ROPA LAVADO,Liquido,ARIEL,3000,NaN
7,201701,10008,20001,0,37,33.18724,33.18724,HC,ROPA LAVADO,Liquido,ARIEL,3000,NaN
8,201701,10009,20001,0,22,15.26725,15.26725,HC,ROPA LAVADO,Liquido,ARIEL,3000,NaN
9,201701,10011,20001,0,5,8.00300,8.00300,HC,ROPA LAVADO,Liquido,ARIEL,3000,NaN


In [7]:
final_dataset[final_dataset['cust_request_tn'] != final_dataset['tn']]

,periodo,customer_id,product_id,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn,cat1,cat2,cat3,brand,sku_size,stock_final
1,201701,10002,20001,0,17,38.68301,35.72806,HC,ROPA LAVADO,Liquido,ARIEL,3000,NaN
195,201702,10010,20001,0,2,45.55551,16.62160,HC,ROPA LAVADO,Liquido,ARIEL,3000,NaN
198,201702,10013,20001,0,7,62.34501,57.42009,HC,ROPA LAVADO,Liquido,ARIEL,3000,NaN
206,201702,10021,20001,0,3,7.01801,6.40240,HC,ROPA LAVADO,Liquido,ARIEL,3000,NaN
220,201702,10037,20001,0,3,4.92492,3.69369,HC,ROPA LAVADO,Liquido,ARIEL,3000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2331742,201911,10017,20127,0,2,15.87338,13.28569,HC,ROPA LAVADO,Liquido,ROPEX2,3000,30.54813
2331750,201911,10034,20127,0,7,36.26120,33.01259,HC,ROPA LAVADO,Liquido,ROPEX2,3000,30.54813
2331868,201912,10017,20127,0,2,12.07586,8.27835,HC,ROPA LAVADO,Liquido,ROPEX2,3000,91.63318
2331941,201910,10052,21087,0,2,0.35850,0.28680,PC,DEOS,Aero,NIVEA,65,4.12060


In [8]:
final_dataset.describe()

,periodo,customer_id,product_id,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn,sku_size,stock_final
count,2.332211e+06,2.332211e+06,2.332211e+06,2.332211e+06,2.332211e+06,2.332211e+06,2.332211e+06,2.332211e+06,984711.000000
mean,2.018085e+05,1.017257e+04,2.038488e+04,1.197576e-02,2.171542e+00,5.216651e-01,5.099546e-01,4.145647e+02,28.420299
std,8.196500e+01,1.421435e+02,3.012938e+02,1.087766e-01,3.666393e+00,3.582281e+00,3.452151e+00,6.733014e+02,78.845498
min,2.017010e+05,1.000100e+04,2.000100e+04,0.000000e+00,1.000000e+00,1.000000e-04,1.000000e-04,1.000000e+00,-13.666560
25%,2.017100e+05,1.005500e+04,2.012600e+04,0.000000e+00,1.000000e+00,1.048000e-02,1.048000e-02,9.000000e+01,1.853560
50%,2.018070e+05,1.013500e+04,2.031600e+04,0.000000e+00,1.000000e+00,4.095000e-02,4.095000e-02,2.200000e+02,7.609590
75%,2.019040e+05,1.026900e+04,2.060100e+04,0.000000e+00,2.000000e+00,1.714400e-01,1.714050e-01,4.500000e+02,23.849910
max,2.019120e+05,1.063700e+04,2.127600e+04,1.000000e+00,9.200000e+01,5.515614e+02,5.478785e+02,1.000000e+04,1562.024480


In [9]:
final_dataset[(final_dataset['plan_precios_cuidados'] != 0) & (~final_dataset['plan_precios_cuidados'].isna())]

,periodo,customer_id,product_id,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn,cat1,cat2,cat3,brand,sku_size,stock_final
65442,201701,10001,20012,1,26,61.05329,61.05329,HC,ROPA ACONDICIONADOR,ACONDICIONADOR,DOWNY,800,NaN
65443,201701,10002,20012,1,11,25.31629,25.31629,HC,ROPA ACONDICIONADOR,ACONDICIONADOR,DOWNY,800,NaN
65444,201701,10003,20012,1,5,39.09093,39.09093,HC,ROPA ACONDICIONADOR,ACONDICIONADOR,DOWNY,800,NaN
65445,201701,10004,20012,1,13,48.54840,36.04350,HC,ROPA ACONDICIONADOR,ACONDICIONADOR,DOWNY,800,NaN
65446,201701,10005,20012,1,6,6.72531,6.72531,HC,ROPA ACONDICIONADOR,ACONDICIONADOR,DOWNY,800,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2295147,201912,10371,20681,1,1,0.00275,0.00275,HC,HOGAR,Gel,MUSCULO,250,14.11118
2295148,201912,10374,20681,1,1,0.01375,0.01375,HC,HOGAR,Gel,MUSCULO,250,14.11118
2295149,201912,10378,20681,1,1,0.01099,0.01099,HC,HOGAR,Gel,MUSCULO,250,14.11118
2295150,201912,10382,20681,1,1,0.00275,0.00275,HC,HOGAR,Gel,MUSCULO,250,14.11118


#### Analizo si puede existir alguna venta de un producto en el mismo periodo que no sea plan_precios_cuidados y otras que si

In [10]:
ventas[(ventas['product_id'] == 20026) & (ventas['periodo'] == 201909) & (ventas['plan_precios_cuidados'] == 0)]

,periodo,customer_id,product_id,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn


In [11]:
def has_both_types(group):
    has_cuidados = (group['plan_precios_cuidados'] != 0).any()
    has_no_cuidados = (group['plan_precios_cuidados'] == 0).any()
    # display(group)
    return has_cuidados and has_no_cuidados

filtered_groups = ventas.groupby(['product_id', 'periodo', 'customer_id']).filter(has_both_types)
filtered_groups

,periodo,customer_id,product_id,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn


Como observo que no existe ningun producto, en ningun periodo, que fuera a su vez precio_cuidoda y que no lo fuera, procedo a modificar esa variable en dataset_final para transformarla en boolean en vez de en suma:

In [12]:
final_dataset['plan_precios_cuidados'] = final_dataset['plan_precios_cuidados'].apply(lambda x: 1 if x > 0 else 0)

In [13]:
final_dataset[(final_dataset['plan_precios_cuidados'] != 0) & (~final_dataset['plan_precios_cuidados'].isna())]

,periodo,customer_id,product_id,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn,cat1,cat2,cat3,brand,sku_size,stock_final
65442,201701,10001,20012,1,26,61.05329,61.05329,HC,ROPA ACONDICIONADOR,ACONDICIONADOR,DOWNY,800,NaN
65443,201701,10002,20012,1,11,25.31629,25.31629,HC,ROPA ACONDICIONADOR,ACONDICIONADOR,DOWNY,800,NaN
65444,201701,10003,20012,1,5,39.09093,39.09093,HC,ROPA ACONDICIONADOR,ACONDICIONADOR,DOWNY,800,NaN
65445,201701,10004,20012,1,13,48.54840,36.04350,HC,ROPA ACONDICIONADOR,ACONDICIONADOR,DOWNY,800,NaN
65446,201701,10005,20012,1,6,6.72531,6.72531,HC,ROPA ACONDICIONADOR,ACONDICIONADOR,DOWNY,800,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2295147,201912,10371,20681,1,1,0.00275,0.00275,HC,HOGAR,Gel,MUSCULO,250,14.11118
2295148,201912,10374,20681,1,1,0.01375,0.01375,HC,HOGAR,Gel,MUSCULO,250,14.11118
2295149,201912,10378,20681,1,1,0.01099,0.01099,HC,HOGAR,Gel,MUSCULO,250,14.11118
2295150,201912,10382,20681,1,1,0.00275,0.00275,HC,HOGAR,Gel,MUSCULO,250,14.11118


In [14]:
# Observo que tambien los Na de plan_precios_cuidados se mapearon a 0
final_dataset[final_dataset['plan_precios_cuidados'].isna()]

,periodo,customer_id,product_id,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn,cat1,cat2,cat3,brand,sku_size,stock_final


#### Defino la variable target

In [15]:
final_dataset.rename(columns={'tn': 'y'}, inplace=True)
final_dataset.to_csv('../Datasets/final_dataset_by_client.csv', sep='\t', index=False)
final_dataset

,periodo,customer_id,product_id,plan_precios_cuidados,cust_request_qty,cust_request_tn,y,cat1,cat2,cat3,brand,sku_size,stock_final
0,201701,10001,20001,0,11,99.43861,99.43861,HC,ROPA LAVADO,Liquido,ARIEL,3000,NaN
1,201701,10002,20001,0,17,38.68301,35.72806,HC,ROPA LAVADO,Liquido,ARIEL,3000,NaN
2,201701,10003,20001,0,17,143.49426,143.49426,HC,ROPA LAVADO,Liquido,ARIEL,3000,NaN
3,201701,10004,20001,0,9,184.72927,184.72927,HC,ROPA LAVADO,Liquido,ARIEL,3000,NaN
4,201701,10005,20001,0,23,19.08407,19.08407,HC,ROPA LAVADO,Liquido,ARIEL,3000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2332206,201912,10430,21214,0,1,0.00179,0.00179,PC,DEOS,RollOn,NIVEA,50,2.69032
2332207,201912,10441,21214,0,1,0.00239,0.00239,PC,DEOS,RollOn,NIVEA,50,2.69032
2332208,201912,10448,21214,0,1,0.00299,0.00299,PC,DEOS,RollOn,NIVEA,50,2.69032
2332209,201912,10505,21214,0,1,0.00179,0.00179,PC,DEOS,RollOn,NIVEA,50,2.69032


#### Analisis nulos

In [16]:
# TODO
final_dataset.isnull().sum()

periodo                        0
customer_id                    0
product_id                     0
plan_precios_cuidados          0
cust_request_qty               0
cust_request_tn                0
y                              0
cat1                           0
cat2                           0
cat3                           0
brand                          0
sku_size                       0
stock_final              1347500
dtype: int64

El stock no parece ser util para los modelos, y es a su vez, la unica variable con valores nulos, por lo que no haremos ninguna tratamiento de momento.

#### Feature Engineering

Agregamos una columna que indica si el mes es cierre de Q, y otra columna que indique la "edad" del producto, considerando la fecha de la primer venta en el dataset

In [17]:
final_dataset['periodo_temp'] = final_dataset['periodo'].astype(str)
final_dataset['close_quarter'] = final_dataset['periodo_temp'].str[-2:].isin(['03', '06', '09', '12']).astype(int)

final_dataset['first_period'] = final_dataset.groupby('product_id')['periodo'].transform('min')
final_dataset['first_period'] = final_dataset['first_period'].astype(str)

final_dataset['age'] = ((final_dataset['periodo_temp'].str[:4].astype(int) - final_dataset['first_period'].str[:4].astype(int)) * 12 +
                        (final_dataset['periodo_temp'].str[-2:].astype(int) - final_dataset['first_period'].str[-2:].astype(int)))


final_dataset.drop(columns=['periodo_temp', 'first_period'], inplace=True)
display(final_dataset.head())

final_dataset.to_csv('../Datasets/final_dataset_by_client.csv', sep='\t', index=False)

,periodo,customer_id,product_id,plan_precios_cuidados,cust_request_qty,cust_request_tn,y,cat1,cat2,cat3,brand,sku_size,stock_final,close_quarter,age
0,201701,10001,20001,0,11,99.43861,99.43861,HC,ROPA LAVADO,Liquido,ARIEL,3000,NaN,0,0
1,201701,10002,20001,0,17,38.68301,35.72806,HC,ROPA LAVADO,Liquido,ARIEL,3000,NaN,0,0
2,201701,10003,20001,0,17,143.49426,143.49426,HC,ROPA LAVADO,Liquido,ARIEL,3000,NaN,0,0
3,201701,10004,20001,0,9,184.72927,184.72927,HC,ROPA LAVADO,Liquido,ARIEL,3000,NaN,0,0
4,201701,10005,20001,0,23,19.08407,19.08407,HC,ROPA LAVADO,Liquido,ARIEL,3000,NaN,0,0


#### Generacion de file para kaggle (para usar en los modelos)

In [18]:
#generar dataset Kaggle
#df.to_csv('../Datasets/df.csv', sep=',', index=False)

#### Formula para calcular el error (para usar en los modelos)

In [19]:
# df_mm['AbsoluteError'] = abs(df_mm['y'] - df_mm['prediction'])
# total_forecast_error = df_mm['AbsoluteError'].sum() / df_mm['y'].sum()